In [1]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import distance_helpers
import numpy as np
import json
import cv2
import operator

In [2]:
inputfiles = ['petfinderPurebred_pca_11063_100_all.pickle','petfinderPurebred_dog_cnn_simple_all.pickle','petfinderPurebred_dog_cnn_inception_all.pickle']

In [3]:
photo_ids = []
photo_indices = []
breedList = dict()
photoId2Index = dict()
with open("small_dog_data.json") as data_file:    
    dog_data = json.load(data_file)
counter = 0
for k,v in dog_data.iteritems():
    nBreeds = float(len(v["breeds"]))
    if len(v["breeds"]) > 1: continue
    breed = v["breeds"][0]
    photos = v["photos"]
    #if len(photos) > 1: print k
    for photo in photos:
        imagename = "cutPetFinder/"+k+"_"+photo["id"]+".jpg"
        img = cv2.imread(imagename)
        if img is None: continue
        photo_ids.append(int(k))
        photo_indices.append(int(photo["id"]))
        photoId2Index[k] = counter
        if not breedList.has_key(breed): breedList[breed] = []
        breedList[breed].append(counter)
        counter += 1
        #break

In [4]:
print len(photo_ids),len(breedList),len(photo_indices)

3301 111 3301


In [13]:
sameDistances = [0.0 for i in range(len(inputfiles))]
sumBreedDistances = [0.0 for i in range(len(inputfiles))]
nGood1 = [0.0 for i in range(len(inputfiles))]
nGood5 = [0.0 for i in range(len(inputfiles))]

print set(photo_indices)
for i,inputfilename in enumerate(inputfiles):
    with open(inputfilename, 'rb') as handle:
        b = pickle.load(handle)
        counter = 0
        coefficients = np.array(b.values())
        similarities = distance_helpers.getDistances(coefficients)
        avgForAll = distance_helpers.getAvgDistance(similarities)
        print inputfilename,coefficients.shape,avgForAll
        print i
        for j,p_id in enumerate(photo_ids):
            #if photo_ids[j-1] == p_id: print j,photo_indices[j],photo_indices[j-1],p_id,photo_ids[j-1]
            if photo_indices[j] < 2: continue
            if p_id != photo_ids[j-1]: continue
            idx = np.array([j-1,j])
            slimSimilarities = similarities[idx[:, None], idx]
            sameAvg = distance_helpers.getAvgDistance(slimSimilarities)
            sameDistances[i] += sameAvg / avgForAll

            thisSimilarities = similarities[j,:]
            sorted_x = sorted(zip(range(len(thisSimilarities)),thisSimilarities), key=operator.itemgetter(1),reverse=True)
            sorted_x5 = sorted_x[1:6]
            ids5 = [photo_ids[ii] for ii,jjj in sorted_x5]
            if p_id in ids5: 
                nGood5[i] += 1
                #print p_id, ids5
            sorted_x1 = sorted_x[1]
            #print sorted_x1
            if p_id == photo_ids[sorted_x1[0]]: nGood1[i] += 1
            counter += 1
        sameDistances[i] /= float(counter)
        nGood1[i] /= float(counter)
        nGood5[i] /= float(counter)

set([1, 2, 3])
petfinderPurebred_pca_11063_100_all.pickle (3301, 100) 0.0953939173752
0
petfinderPurebred_dog_cnn_simple_all.pickle (3301, 120) 0.00910911050256
1
petfinderPurebred_dog_cnn_inception_all.pickle (3301, 120) 0.045489572719
2


In [14]:
print sameDistances
print nGood1
print nGood5

[1.1010955643403244, 1.5892520615025167, 2.3807266185730307]
[0.03125, 0.04318181818181818, 0.028409090909090908]
[0.04375, 0.10965909090909091, 0.06534090909090909]
